In [ ]:
!pip install optuna
!pip install xgboost
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [2]:
import  numpy as np
import os
import cv2
from tqdm import tqdm
import optuna
from matplotlib import pyplot as plt
import seaborn as sns
from skimage.feature import hog
from google.colab import drive
from cuml.decomposition import PCA
from cuml.svm import SVC
from cuml.linear_model import LogisticRegression
from cuml.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!apt-get install p7zip-full

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
!7z x /content/drive/MyDrive/archive.zip.001 -o/content/datasets

In [6]:
data_dir = "datasets"

categories = ["NonDemented", "VeryMildDemented", "MildDemented", "ModerateDemented"]

class_count = {}
for class_name in categories:
    class_dir = os.path.join(data_dir, class_name, class_name)
    num_images = len(os.listdir(class_dir))
    class_count[class_name] = num_images
print(class_count)

{'NonDemented': 63560, 'VeryMildDemented': 13800, 'MildDemented': 5184, 'ModerateDemented': 376}


In [7]:
fig, axes = plt.subplots(1, 4, figsize=(15, 5))
for i, class_name in enumerate(categories):
    class_dir = os.path.join(data_dir, class_name, class_name)
    sample_image_path = os.path.join(class_dir, os.listdir(class_dir)[0])
    img = cv2.imread(sample_image_path)
    axes[i].imshow(img)
    axes[i].set_title(class_name)
    axes[i].axis("off")


In [8]:
image_size = 64

x = []
y = []

valid_extensions = (".jpg", ".jpeg", ".png")

for label, category in enumerate(categories):
    path = os.path.join(data_dir, category, category)
    print(f"Reading category: {category}, Label: {label}")

    for image_name in tqdm(os.listdir(path)):
        if not image_name.lower().endswith(valid_extensions):
            continue

        image_path = os.path.join(path, image_name)

        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (image_size, image_size))

        x.append(img)
        y.append(label)

x = np.array(x)
y = np.array(y)

print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Reading category: NonDemented, Label: 0


100%|██████████| 63560/63560 [04:24<00:00, 240.47it/s]


Reading category: VeryMildDemented, Label: 1


100%|██████████| 13800/13800 [00:51<00:00, 266.01it/s]


Reading category: MildDemented, Label: 2


100%|██████████| 5184/5184 [00:21<00:00, 244.24it/s]


Reading category: ModerateDemented, Label: 3


100%|██████████| 376/376 [00:01<00:00, 268.87it/s]


Shape of x: (82916, 64, 64)
Shape of y: (82916,)


In [9]:
plt.figure(figsize=(15, 5))
sns.histplot(y, bins=len(categories))
plt.title("class distribution")
plt.xlabel("Classes")
plt.xticks([0, 1, 2, 3], labels=categories)
plt.ylabel("Count")

Text(0, 0.5, 'Count')

In [10]:
x_cupy = x / 255.0
y_cupy = y

In [11]:
def extract_hog_features(image):
    features = hog(image, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), visualize=False,)
    return features

x_hog_features = np.array([extract_hog_features(img) for img in x_cupy])

pca = PCA(n_components=min(100, x_hog_features.shape[0]))
x_pca = pca.fit_transform(x_hog_features)

print( x_hog_features.shape)
print(x_pca.shape)


(82916, 1764)
(82916, 100)


In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_pca, y_cupy, test_size=0.2, random_state=42)

print(x_train.shape)
print(y_train.shape)

(66332, 100)
(66332,)


In [ ]:
def optimize_svm(trial):
    c = trial.suggest_categorical('C', [0.01, 0.1, 1, 10])

    model = SVC(C=c, random_state=42)
    accuracy = cross_val_score(model, x_train, y_train, cv=5, scoring='accuracy').mean()
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(optimize_svm, n_trials=10)

best_param_svm = study.best_params

print(best_param_svm)

In [21]:
best_svm = SVC(**best_param_svm, probability=True)
best_svm.fit(x_train, y_train)

y_pred = best_svm.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9914978292329957


In [ ]:
def optimize_rf(trial):
  n_estimators = trial.suggest_int('n_estimators', 50, 150, step=10)
  max_dept = trial.suggest_int('max_depth', 5, 10, 15)
  min_samples_split = trial.suggest_int('min_samples_split', 3, 5, 7)

  model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_dept, min_samples_split=min_samples_split, random_state=42)

  accuracy = cross_val_score(model, x_train, y_train, cv=5, scoring='accuracy').mean()
  return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(optimize_rf, n_trials=10)

best_param_rf = study.best_params

print(best_param_rf)

In [16]:
best_rf = RandomForestClassifier(**best_param_rf)
best_rf.fit(x_train, y_train)

y_pred = best_rf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.7657983598649301


In [ ]:
def optimize_xgb(trial):
  n_estimators = trial.suggest_int('n_estimators', 20, 70, step=10)
  learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1)
  max_depth = trial.suggest_int('max_depth', 5, 10, 15)

  model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)

  accuracy = cross_val_score(model, x_train, y_train, cv=3, scoring='accuracy').mean()
  return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(optimize_xgb, n_trials=10)

best_param_xgb = study.best_params

print(best_param_xgb)

In [19]:
best_xgb = XGBClassifier(**best_param_xgb, tree_method = "hist", device = "cuda")
best_xgb.fit(x_train, y_train)

y_pred = best_xgb.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8089122045344911


In [22]:
base_models = [
    ('svm', best_svm),
    ('rf', best_rf),
    ('xgb', best_xgb)
]

meta_model = LogisticRegression()

stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

stacking_model.fit(x_train, y_train)
y_pred_stack = stacking_model.predict(x_test)

print(classification_report(y_test, y_pred_stack))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     12646
           1       0.99      0.97      0.98      2840
           2       0.99      0.97      0.98      1014
           3       0.98      0.98      0.98        84

    accuracy                           0.99     16584
   macro avg       0.99      0.98      0.98     16584
weighted avg       0.99      0.99      0.99     16584

